## **Generar archivo para recomendaciones**

1. **Carga de datos**: El proceso comienza cargando el conjunto de datos 'data_games_clean.csv', que contiene información sobre los juegos de Steam. Este archivo incluye columnas como el nombre del juego, sus especificaciones y un identificador único para cada juego.

2. **Preprocesamiento de datos**: Se seleccionan las columnas relevantes del conjunto de datos, que son las especificaciones y el identificador único del juego. El identificador único se convierte a tipo entero para facilitar su manipulación.

3. **Vectorización de texto**: Las especificaciones de cada juego se procesan utilizando CountVectorizer, una técnica de vectorización de texto que convierte el texto en una representación numérica que puede ser utilizada por algoritmos de aprendizaje automático.

4. **Cálculo de similitud**: Se utiliza la similitud del coseno para calcular la similitud entre las especificaciones de todos los pares de juegos en el conjunto de datos. Este cálculo crea una matriz de similitud que indica cuán similares son los juegos entre sí en función de sus especificaciones.

5. **Generación de recomendaciones**: Se define una función que toma un juego como entrada y devuelve una lista de los cinco juegos más similares a él, según la matriz de similitud calculada anteriormente. Estas recomendaciones se basan en las especificaciones de los juegos, lo que permite identificar juegos similares en función de sus características.

6. **Creación del archivo CSV de recomendaciones**: Se aplica la función de recomendación a cada juego en el conjunto de datos y se crea una nueva columna llamada 'Recomendaciones' que contiene las listas de juegos recomendados. Las columnas innecesarias se eliminan para reducir el tamaño del archivo final.

7. **Almacenamiento del archivo CSV**: Finalmente, el DataFrame resultante se guarda como un archivo CSV llamado 'feel.csv' en la ubicación especificada ('C:\Users\pc-admin\Documents\ML_RecSys_Model\SistemReco'). Este archivo contiene las recomendaciones de juegos basadas en similitudes de especificaciones.

Este proceso permite a los usuarios descubrir nuevos juegos similares a sus preferidos en función de las características y especificaciones de los juegos existentes en el conjunto de datos de Steam.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar el conjunto de datos
steam_games = pd.read_csv(r'C:\Users\pc-admin\Documents\ML_RecSys_Model\Data_ETL\data_games_clean.csv')

# Seleccionar las columnas necesarias
df = steam_games.loc[:, ["specs", "id", "app_name"]]

# Convertir 'id' a tipo entero
df["id"] = df["id"].astype(int)

# Limpiar la columna 'specs'
df['specs'] = df['specs'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace("'", ''))

# Crear un vectorizador de texto
cv = CountVectorizer()
vectores = cv.fit_transform(df['specs']).toarray()

# Calcular la similitud del coseno entre vectores
similitud = cosine_similarity(vectores)

# Generar una función para obtener recomendaciones por título
def recomendacion(juego):
    # Se busca el índice del juego en el DataFrame original (df). Este índice es utilizado para acceder a la fila correspondiente en la matriz de similitud.
    indice_juego = df[df["id"] == juego].index[0]
    
    # Cálculo de Similitudes: Se obtienen las distancias de similitud entre el juego de entrada y todos los demás juegos en el conjunto de datos. 
    distances = similitud[indice_juego]
    
    # Las distancias se ordenan de manera descendente, y se seleccionan los cinco juegos más similares (excluyendo el juego de entrada)
    lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    # Salida. Devuelve la lista de títulos recomendados.
    recommended_titles = [df.iloc[i[0]]['app_name'] for i in lista_juegos]
    
    return recommended_titles

# Aplicar la función a la columna 'id' y crear una nueva columna 'Recomendaciones'
df['Recomendaciones'] = df['id'].apply(recomendacion)

# Eliminar columnas innecesarias para disminuir el tamaño del archivo de salida
df.drop(columns=['app_name', 'specs'], inplace=True)

# Guardar el DataFrame como archivo CSV
df.to_csv(r'C:\Users\pc-admin\Documents\ML_RecSys_Model\SistemReco\feel.csv', index=False)
